# タイトル: アンケート調査データのETL・正規化パイプライン
説明: Webから取得したアンケートJSONデータをDatabricks上でBronze/Silverテーブルに整備・正規化し、分析やAI処理に活用できる形に変換するノートブックです。

### 1. パラメータ設定
データ格納先のカタログ・スキーマ・ボリューム名や、ダウンロード元URL、保存先パスなどを定義しています。

In [0]:
# ===== パラメータ（必要に応じて変更） =====
CATALOG = "hiroshi"
SCHEMA  = "survey_analysis_simple"
VOLUME  = "raw_data"

# 例：公開URL（JSON Lines）をここに設定
SOURCE_URL = "https://raw.githubusercontent.com/hiouchiy/voc_analysis_demo/refs/heads/main/data/survey_responses.jsonl"

# UC Volume 上の保存先（ファイル名は任意）
DST_DIR  = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/landing"
DST_FILE = f"{DST_DIR}/survey_responses.jsonl"

# LLMエンドポイント（ai_extract/ai_queryで使用）
LLM_ENDPOINT = "databricks-claude-3-7-sonnet"   # 例。自分のFMエンドポイント名に置換

# テーブル名
BRONZE_TBL = "bronze_survey_responses"
SILVER_TBL = "silver_survey_responses_base"

### 2. UCオブジェクトの作成・選択
指定したカタログ・スキーマ・ボリュームが存在しなければ作成し、以降の操作対象として選択します。

In [0]:
# UCオブジェクト作成
spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG}")
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA}")
spark.sql(f"USE {SCHEMA}")
spark.sql(f"CREATE VOLUME IF NOT EXISTS {VOLUME} COMMENT 'Landing for survey raw JSON'")

### 3. データのダウンロードと保存
Web上のアンケートデータ（JSON Lines）をダウンロードし、一時ファイル経由でUC Volumeに保存します。
※アウトバウンド通信が制限されている場合は、手動アップロードが必要です。

In [0]:
# 保存先ディレクトリ準備（Volumesパスはdbutils.fsで作成）
dbutils.fs.mkdirs(DST_DIR)

# --- Webからダウンロード → UC Volumeへ書き込み ---
# 注意：Workspaceのアウトバウンド通信が制限されている場合は失敗します。その場合はローカルからVolumeへ直接アップロードしてください（サイドバーの"Volumes" > "Upload"）。
import requests

resp = requests.get(SOURCE_URL, timeout=60)
resp.raise_for_status()
content = resp.content  # bytes

# 一時ファイルに保存してから、dbutils.fs.putでVolumeへ書き込み
import tempfile, os
tmp = tempfile.mktemp(suffix=".jsonl")
with open(tmp, "wb") as f:
    f.write(content)

with open(tmp, "r", encoding="utf-8") as f:
    dbutils.fs.put(DST_FILE, f.read(), overwrite=True)

os.remove(tmp)
print(f"Downloaded to: {DST_FILE}")

### 4. データの読み込みと型整形
保存したJSON LinesファイルをSpark DataFrameで読み込み、日付や真偽値など必要な型変換を行います。

#### 1) Bronze：UC Volume → Delta（JSON Lines想定）
アンケートの生データ（JSON Lines形式）をDatabricksのBronzeテーブルとして保存する処理です。

主な流れは以下の通りです。

1. カタログ・スキーマの選択
Databricks Unity Catalogの対象カタログ・スキーマを選択します。

1. JSON Linesデータの読み込み
Volumesに保存されたJSON Linesファイル（1行1レコード）をSpark DataFrameとして読み込みます。

1. 型整形
提出日時（submitted_at）をタイムスタンプ型に変換し、drives_weekly列をboolean型に変換します。

1. Deltaテーブル（Bronzeテーブル）への保存
整形したDataFrameをDelta形式でテーブル（bronze_survey_responses）として上書き保存します。
Bronzeテーブルは生データの原形保持が原則です。

1. テーブルの最適化
OPTIMIZEコマンドでDeltaテーブルのパフォーマンスを向上させます。

In [0]:
# ---- JSON Lines 読み込んで、表示させてみましょう ----
df_bronze = (
    spark.read
         .option("multiLine", "false")  # JSON Lines想定（1行=1レコード）
         .json(DST_FILE)
)

# ※配列JSONを読みたい場合は上記を以下に変更
# df_bronze = (
#     spark.read
#          .option("multiLine", "true")   # 配列JSON [{},{}...]
#          .json(DST_FILE)
# )

display(df_bronze)

In [0]:
# df_bronzeの中のsubmitted_at, drives_weeklyをタイムスタンプ型、Boolean型にそれぞれ型を定義し、Delta Tableを新規作成するコードを書いてください。なお、同名のTableが存在する場合は全体を上書きしてOKです。

In [0]:
# 正解コードはこちら
# いきなり見るのはやめましょう

from pyspark.sql import functions as F

# 必要最低限の型整形（submitted_at, drives_weekly）
df_bronze = (
    df_bronze
      .withColumn("submitted_at", F.to_timestamp("submitted_at", "yyyy-MM-dd HH:mm:ss"))
      .withColumn("drives_weekly", F.col("drives_weekly").cast("boolean"))
)

# Deltaに上書き作成（Bronzeは原形保持が原則。既存があればappendにしても良い）
(df_bronze
 .write
 .format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable(BRONZE_TBL)
)

# spark.sql(f"OPTIMIZE {BRONZE_TBL}")
print(f"Bronze table: {CATALOG}.{SCHEMA}.{BRONZE_TBL}")

#### 2) Silver

Bronzeテーブルのアンケートデータを「型整備・正規化」してSilverテーブルに変換・保存する処理です。

主な処理内容は以下の通りです。

1. Bronzeテーブルの読み込み
Deltaテーブル（Bronze）からデータを取得します。

1. 列ごとの型変換・値の正規化
    - id, submitted_at, drives_weeklyなどを適切な型（bigint, timestamp, boolean）に変換
    - age_group, gender, region, occupationなどのカテゴリ値を許容リストで正規化（不正値はデフォルト値に）
    - satisfaction（満足度）を1～5の整数型に変換
    - fav_alcohols, health_reasonsを配列型に整形
    - submitted_date, submitted_hourを抽出

1. id重複の解消
同じidが複数ある場合は、最新のsubmitted_atのみを残します。

1. Silverテーブルへの保存
正規化したデータをDelta形式でSilverテーブルに上書き保存します。

1. テーブルプロパティ設定・最適化
テーブルの用途説明（purpose, description）を付与し、OPTIMIZEでパフォーマンスを向上させます。

In [0]:
# ===== Silver: 型整備・正規化（Bronzeに存在する列のみ） =====
from pyspark.sql import functions as F

df = spark.table(BRONZE_TBL)

# ユーティリティ：カンマ区切り → 正規化配列
def to_clean_array(col):
    return F.expr(
        "filter(array_distinct(transform(split(coalesce({c}, ''), ','), x -> trim(lower(x)))), x -> x <> '')"
        .format(c=col)
    )

# 想定コード値（存在する列のみ）
allowed_age      = ["u19","20s","30s","40s","50s","60plus"]
allowed_gender   = ["male","female","other","no_answer"]
allowed_region   = ["hokkaido_to_hoku","kanto","chubu","kinki","chugoku_shikoku","kyushu_okinawa","overseas","no_answer"]
allowed_occ      = ["company_employee","self_employed","student","homemaker","other","no_answer"]
allowed_drink    = ["daily","weekly","monthly","never"]
allowed_nonalc   = ["first_time","sometimes","weekly"]
allowed_price    = ["under199","200_249","250_299","300_349","350plus"]
allowed_intent   = ["def_buy","maybe","unsure","no"]

def norm_enum(col, allowed, default="unknown"):
    return (
        F.when(F.col(col).isNull() | (F.trim(F.col(col)) == ""), F.lit(default))
         .otherwise(
             F.when(F.lower(F.col(col)).isin([a.lower() for a in allowed]),
                    F.lower(F.col(col))).otherwise(F.lit(default))
         )
    )

# 型整備・正規化（Bronze列のみ）
df_silver = (
    df
    .withColumn("id",               F.col("id").cast("bigint"))
    .withColumn("submitted_at",     F.col("submitted_at").cast("timestamp"))
    .withColumn("drives_weekly",    F.col("drives_weekly").cast("boolean"))
    .withColumn("age_group",        norm_enum("age_group", allowed_age))
    .withColumn("gender",           norm_enum("gender", allowed_gender,  "no_answer"))
    .withColumn("region",           norm_enum("region", allowed_region,  "no_answer"))
    .withColumn("occupation",       norm_enum("occupation", allowed_occ, "other"))
    .withColumn("drinking_freq",    norm_enum("drinking_freq", allowed_drink))
    .withColumn("nonalc_freq",      norm_enum("nonalc_freq", allowed_nonalc))
    .withColumn("price_band",       norm_enum("price_band", allowed_price, "unknown"))
    .withColumn("purchase_intent",  norm_enum("purchase_intent", allowed_intent))
    .withColumn("satisfaction_int",
        F.when(F.col("satisfaction").rlike("^[1-5]$"),
               F.col("satisfaction").cast("int")).otherwise(F.lit(None).cast("int"))
    )
    .withColumn("fav_alcohols_arr", to_clean_array("fav_alcohols"))
    .withColumn("health_reasons_arr", to_clean_array("health_reasons"))
    .drop("fav_alcohols", "health_reasons", "satisfaction")
)

# 万一の id 重複は最新 submitted_at を採用
from pyspark.sql.window import Window
w = Window.partitionBy("id").orderBy(F.col("submitted_at").desc_nulls_last())
df_silver = (
    df_silver
    .withColumn("_rn", F.row_number().over(w))
    .filter(F.col("_rn") == 1)
    .drop("_rn")
)

# Delta に上書き
(df_silver.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(SILVER_TBL)
)

spark.sql(f"ALTER TABLE {SILVER_TBL} SET TBLPROPERTIES (purpose='silver_base', description='Type-normalized survey data (no ai_extract)')")
# spark.sql(f"OPTIMIZE {SILVER_TBL}")

print(f"Silver(base) table ready: {CATALOG}.{SCHEMA}.{SILVER_TBL}")

#### 3) Gold
アンケート回答の自由記述（free_comment）からAIモデルを使って情報抽出する一連の処理を行っています。

1. **抽出プロンプトの生成 (_nlp_promptビュー)**
   - 各回答のfree_commentに対し、AIモデルに渡すプロンプト（指示文）を作成します。
   - ポジティブ要素、ネガティブ要素、飲用シーンを簡潔かつサマライズして抽出するよう指示しています。
   - NULL値対策として`COALESCE(free_comment, '')`で空文字に変換しています。

2. **AIモデルによる情報抽出 (_nlp_ai_rawビュー)**
   - Databricksの`ai_query`関数を使い、指定したモデル（例: 'databricks-gpt-oss-120b'）にプロンプトを渡して構造化情報を抽出します。
   - 抽出結果はSTRUCT型で返され、positive_points, negative_feedbacks, scenesが含まれます。

3. **抽出結果のビュー化 (gold_survey_responses_nlpビュー)**
   - 元データ（silver_survey_responses_base）とAI抽出結果をidで結合し、抽出情報を新しい列として展開します。

In [0]:
%sql
-- 1) 抽出プロンプト（できるだけ簡潔＆安定）
--    - % や複雑な装飾を避け、文字列はシンプルに
CREATE OR REPLACE TEMP VIEW _nlp_prompt AS
SELECT
  id,
  free_comment,
  CONCAT(
    '以下の日本語の感想文から情報抽出してください。',
    '\n\n[出力仕様]',
    '\n- positive_points: 感想の中のポジティブな意見のうち、最も強調されているものを短文で一つ抽出してください（例: 香りがよい、後味がすっきり）。同様の内容などは一つの文章にまとめ上げるなど、サマライズして、冗長さを取り除いて、簡潔にしてください。そして、日本語で出力してください。句読点などは不要です。ただし、シーンに関するポジティブな言及はscenesに含め、positive_pointsには含めないでください。',
    '\n- negative_feedbacks: ネガティブな意見、または、要望や提案のうち、最も強調されているものを短文で一つ抽出してください。同様の内容などは一つの文章にまとめ上げるなど、サマライズして、冗長さを取り除いて、簡潔にしてください。原文の言葉表現がカジュアルな場合は、同じ意味のビジネスライクな表現に書き直してください。そして、日本語で出力してください。句読点などは不要です。',
    '\n- scenes: 飲用シーン（例: 夕食、風呂上がり、運転前、スポーツ後 など）のうち、最も強調されているものを単語で一つ出力してください。同様の内容などは一つのワードにまとめ上げるなど、サマライズして、冗長さを取り除いて、簡潔にしてください。そして、日本語で出力してください。句読点などは不要です。',
    '\n\n[出力形式]',
    '\n構造化: positive_point, negative_feedback, scene',
    '\n\n[対象の感想文]\n',
    COALESCE(free_comment, '')
  ) AS prompt
FROM silver_survey_responses_base
WHERE COALESCE(TRIM(free_comment), '') <> '';

-- 2) ai_query() 実行
--    - 自分のモデルエンドポイント名に置き換えてください（例: 'databricks-claude-3-7-sonnet' など）
--    - responseFormat を STRUCT で指定して、そのまま列展開できるようにします
CREATE OR REPLACE TEMP VIEW _nlp_ai_raw AS
SELECT
  p.id,
  ai_query(
    'databricks-gpt-oss-120b',
    p.prompt,
    responseFormat => 'STRUCT<
      feedback_extraction:STRUCT<
        positive_point: STRING,
        negative_feedback: STRING,
        scene: STRING
      >
    >'
  ) AS out
FROM _nlp_prompt p;

-- 3) サブSTRUCT配下を正しく参照する
CREATE OR REPLACE TEMP VIEW gold_survey_responses_nlp AS
SELECT
  b.id,
  b.submitted_at,
  b.age_group,
  b.gender,
  b.occupation,
  b.region,
  b.drives_weekly,
  b.drinking_freq,
  b.fav_alcohols_arr,
  b.nonalc_freq,
  b.health_reasons_arr,
  b.purchase_intent,
  b.price_band,
  b.satisfaction_int,
  b.free_comment,
  r.out:positive_point     AS positive_point,
  r.out:negative_feedback  AS negative_feedback,
  r.out:scene              AS scene
FROM silver_survey_responses_base b
LEFT JOIN _nlp_ai_raw r USING (id);

SELECT * FROM gold_survey_responses_nlp LIMIT 10;

4. **AIモデルによる文章分類結果を追加してテーブル化 (gold_survey_responses_finalテーブル)**
   - AIモデルを使ってポジティブコメントとネガティブコメントを分類し、その結果を新規列として追加した後Tableとして保存。

In [0]:
%sql

-- 1. Store the guideline text in a table to avoid session variable size limits
CREATE OR REPLACE TEMP VIEW guideline_text_view AS
SELECT '''
以下のガイドに従って、コメントの内容を18個のクラスのいずれかに分類してください。ただし、コメントが空白の場合はNoneを出力してください。

# 指標分類ガイドライン
## 領域1: Sensory Quality ― 製品そのものの味わい
### Appearance
- 定義：色調、透明度、泡立ち、グラス映えなど外観に関する評価
- キーワード例：「黄金色」「濁りが少ない」「泡がすぐ消える」
- 除外例：缶の見た目やデザインは → `PackagingDesign`

### Aroma
- 定義：香り（ホップ、モルト、果実香、オフフレーバーなど）に関する記述
- キーワード例：「トロピカルな香り」「ホップが弱い」「香りが薬品っぽい」
- 除外例：味や後味の話 → `Flavor` / `DrinkabilityAftertaste`

### Flavor
- 定義：甘味、苦味、酸味、コク、バランス、余韻など味覚の構成
- キーワード例：「麦の甘み」「苦みが強い」「人工甘味料っぽい味」
- 除外例：香りに関する話 → `Aroma`

### MouthfeelCarbonation
- 定義：炭酸の強さ、粘度、ボディ感、舌触りなどの口当たり
- キーワード例：「クリーミーな泡」「ガス弱め」「軽い口当たり」
- 除外例：「飲みやすさ」は → `DrinkabilityAftertaste`

### DrinkabilityAftertaste
- 定義：飲みやすさ、ゴクゴク感、後味のキレや残り方
- キーワード例：「ゴクゴク飲める」「後味がべたつく」「キレがある」
- 除外例：味覚の詳細な記述 → `Flavor`

## 領域2: Functional Health & Safety ― “飲める理由” を決める要素
### ABV
- 定義：アルコール度数、0.00% 表記、法的基準
- キーワード例：「正真正銘0.00%」「0.3%なのが残念」
- 除外例：「酔わないから安心」→ `SocialAcceptability` 併用可

### CaloriesCarbs
- 定義：カロリー、糖質量、栄養成分表示の明確さ
- キーワード例：「100mlで8kcal」「糖質ゼロ」「栄養表示をもっと目立たせて」
- 除外例：甘さの味覚は → `Flavor`

### AdditivesAllergen
- 定義：保存料、香料、人工甘味料、アレルゲン（例：グルテン）
- キーワード例：「保存料不使用」「大麦アレルギー注意」
- 除外例：素材自体の良さや産地 → `Ingredients`

### Ingredients
- 定義：使用されている素材そのものの質、産地、自然由来かどうか
- キーワード例：「国産モルト使用」「○○渓谷の水」
- 除外例：カロリーや添加物の話 → それぞれの該当項目へ

## 領域3: Practical Usability ― 使い勝手・シーン適合性
### PackagingDesign
- 定義：缶・瓶の色、手触り、ロゴ、開けやすさなど外装デザイン
- キーワード例：「マットな質感」「プルタブが硬い」「ラベルが可愛い」
- 除外例：内容量やパック数の話 → `PackServingFormat`

### PackServingFormat
- 定義：容量バリエーション（250ml/350mlなど）やパッケージ形式（6本パックなど）
- キーワード例：「250mlスリム缶」「6本パックが欲しい」
- 除外例：価格そのものへの評価 → `PriceValue`

### ShelfLifeStorage
- 定義：賞味期限、常温保存の可否、開栓後の劣化速度
- キーワード例：「常温保存OK」「開けたらすぐ劣化する」
- 除外例：外装の物理的な強度や見た目 → `PackagingDesign`

## 領域4: Economic & Availability ― 手に取りやすさ
### PriceValue
- 定義：価格、コスパ、値ごろ感に対する評価
- キーワード例：「280円は高い」「この味でこの値段は安い」
- 除外例：パッケージサイズ・構成 → `PackServingFormat`

### Availability
- 定義：購入のしやすさ、流通チャネル（コンビニ、ECなど）
- キーワード例：「どこにも売ってない」「オンライン限定」
- 除外例：値段が高い／安い → `PriceValue`

### PromotionVisibility
- 定義：広告・店頭POP・販促イベントなどの目立ち方や訴求力
- キーワード例：「POPが目立つ」「試飲イベント良かった」
- 除外例：ブランドの歴史や理念 → `BrandImageStory`

## 領域5: Brand & Emotional Perception ― 心理的・社会的価値
### BrandImageStory
- 定義：ブランドの世界観、理念、クラフト感、物語性
- キーワード例：「クラフト感がある」「創業者の想いに共感」
- 除外例：具体的な環境配慮活動 → `SustainabilityEthics`

### SocialAcceptability
- 定義：運転中、妊娠中、仕事中などでも「飲んでOK」と思える感覚
- キーワード例：「運転前に飲めて安心」「会議中でも飲める」
- 除外例：ABVの数値のみの言及 → `ABV`（併用可）

### SustainabilityEthics
- 定義：環境配慮、リサイクル素材、公正取引、企業倫理
- キーワード例：「再生アルミ缶」「CO₂削減」「ビーガン対応」
- 除外例：ブランドのイメージ全般 → `BrandImageStory`
''' AS guideline_text;

-- =========================================
-- 1) 分類プロンプトを行ごとに作成
--    - 文字化け回避のため、できるだけシンプルなASCIIと箇条書き
--    - 18カテゴリを厳密名で列挙（どれか1つを返す）
-- =========================================
CREATE OR REPLACE TEMP VIEW _fb_cls_prompt AS
SELECT
  r.*,      -- 1件の短文
  CONCAT(
    g.guideline_text,
    '\n\n【ポジティブなコメント】',
    '\n', COALESCE(r.positive_point, '')
  ) AS prompt_for_positive_point,
  CONCAT(
    g.guideline_text,
    '\n\n【ネガティブなコメント】',
    '\n', COALESCE(r.negative_feedback, '')
  ) AS prompt_for_negative_feedback
FROM gold_survey_responses_nlp r
CROSS JOIN guideline_text_view g;

-- =========================================
-- 2) ai_query() 実行
--    - モデルエンドポイントは自分の環境のものに置換してください
--    - STRUCTスキーマで受け取り、そのまま列展開可能
-- =========================================
CREATE OR REPLACE TEMP VIEW _fb_cls_raw AS
SELECT
  p.*,
  ai_query(
    'databricks-gpt-oss-120b',        -- ←あなたのエンドポイント名に変更
    p.prompt_for_positive_point,
    responseFormat => 'STRUCT<classification:STRUCT<category:STRING, confidence:DOUBLE>>'
  ) AS positive_point_ai_out,
  ai_query(
    'databricks-gpt-oss-120b',        -- ←あなたのエンドポイント名に変更
    p.prompt_for_negative_feedback,
    responseFormat => 'STRUCT<classification:STRUCT<category:STRING, confidence:DOUBLE>>'
  ) AS negative_feedback_ai_out
FROM _fb_cls_prompt p;

-- =========================================
-- 3) 結果を正規の列に展開
-- =========================================
CREATE OR REPLACE TABLE gold_survey_responses_final AS
SELECT
  r.* EXCEPT(prompt_for_positive_point, prompt_for_negative_feedback, positive_point_ai_out, negative_feedback_ai_out),
  positive_point_ai_out:category   AS positive_point_category,
  positive_point_ai_out:confidence AS positive_point_confidence,
  negative_feedback_ai_out:category   AS negative_feedback_category,
  negative_feedback_ai_out:confidence AS negative_feedback_confidence
FROM _fb_cls_raw r;

SELECT * FROM gold_survey_responses_final LIMIT 10;

In [0]:
%sql

-- ===== テーブル全体コメント =====
COMMENT ON TABLE gold_survey_responses_final IS '
アンケート回答データ。加えて、さまざまなデータ加工を行い、いくつかの分析用のデータを追加したもの。';

-- ===== gold_survey_responses_final =====
COMMENT ON COLUMN gold_survey_responses_final.id IS '回答一意識別子 (primary key)；feedback テーブルとの参照キー';
COMMENT ON COLUMN gold_survey_responses_final.submitted_at IS '回答提出日時（タイムスタンプ）';
COMMENT ON COLUMN gold_survey_responses_final.age_group IS '年代グループ（例: u19, 20s, 30s, 40s, 50s, 60plus）';
COMMENT ON COLUMN gold_survey_responses_final.gender IS '性別カテゴリ：male, female, other, no_answer';
COMMENT ON COLUMN gold_survey_responses_final.occupation IS '職業カテゴリ（例: company_employee, student, homemaker, other）';
COMMENT ON COLUMN gold_survey_responses_final.region IS '地域カテゴリ（hokkaido_to_hoku, kanto, chubu, kinki, chugoku_shikoku, kyushu_okinawa, overseas, no_answer）';
COMMENT ON COLUMN gold_survey_responses_final.drives_weekly IS '週1 回以上運転するかどうか（真偽値）';
COMMENT ON COLUMN gold_survey_responses_final.drinking_freq IS '普段の飲酒頻度：daily, weekly, monthly, never';
COMMENT ON COLUMN gold_survey_responses_final.fav_alcohols_arr IS '通常好む酒類（文字列、複数可）（ARRAY<STRING>）';
COMMENT ON COLUMN gold_survey_responses_final.nonalc_freq IS 'ノンアル飲用頻度：first_time, sometimes, weekly';
COMMENT ON COLUMN gold_survey_responses_final.health_reasons_arr IS 'health_reasons を配列化したもの（ARRAY<STRING>）';
COMMENT ON COLUMN gold_survey_responses_final.purchase_intent IS '購入意向カテゴリ：def_buy, maybe, unsure, no';
COMMENT ON COLUMN gold_survey_responses_final.price_band IS '許容価格帯カテゴリ（例: under199, 200_249, 250_299, 300_349, 350plus）';
COMMENT ON COLUMN gold_survey_responses_final.satisfaction_int IS '満足度を整数型に変換したもの（1～5 の INT）';
COMMENT ON COLUMN gold_survey_responses_final.free_comment IS '自由記述による感想文';
COMMENT ON COLUMN gold_survey_responses_final.positive_point IS 'AI 抽出によるポジティブ文句群（ARRAY<STRING>）';
COMMENT ON COLUMN gold_survey_responses_final.negative_feedback IS 'AI 抽出によるネガティブ・改善要望文句群（ARRAY<STRING>）';
COMMENT ON COLUMN gold_survey_responses_final.scene IS '飲用シーン語句（ARRAY<STRING>）';
COMMENT ON COLUMN gold_survey_responses_final.positive_point_category IS '
positive_pointの分類カテゴリ：以下 18 種類のいずれか。  
1. Appearance：外観・見た目（色・透明度・泡質）  
2. Aroma：香り（ホップ香・モルト香・果実香・オフ香）  
3. Flavor：味覚（甘味・苦味・酸味・コク・バランス）  
4. MouthfeelCarbonation：口当たり・炭酸感（泡・ガス強度・舌触り）  
5. DrinkabilityAftertaste：飲みやすさ・後味（ゴクゴク・キレ・残留感）  
6. ABV：アルコール度数への言及（例: “0.00%”）  
7. CaloriesCarbs：カロリー・糖質関係の言及  
8. AdditivesAllergen：添加物・香料・アレルゲンの言及  
9. Ingredients：原材料・素材・産地・自然由来  
10. PackagingDesign：包装・ラベル・外装デザイン  
11. PackServingFormat：容量・パッケージ形態（缶・瓶・パック数）  
12. ShelfLifeStorage：保存性・賞味期限・開封後品質  
13. PriceValue：価格・コスパ・値ごろ感  
14. Availability：流通性・店舗・EC 販売可能性  
15. PromotionVisibility：販促・広告・POP・訴求力  
16. BrandImageStory：ブランドイメージ・理念・物語性  
17. SocialAcceptability：運転中・妊娠中などでも飲める許容感  
18. SustainabilityEthics：環境配慮・サステナビリティ・倫理性';
COMMENT ON COLUMN gold_survey_responses_final.positive_point_confidence IS 'positive_point_category に対するConfidence Score。分類信頼度（0.0～1.0）';
COMMENT ON COLUMN gold_survey_responses_final.negative_feedback_category IS '
negative_feedbackの分類カテゴリ：以下 18 種類のいずれか。  
1. Appearance：外観・見た目（色・透明度・泡質）  
2. Aroma：香り（ホップ香・モルト香・果実香・オフ香）  
3. Flavor：味覚（甘味・苦味・酸味・コク・バランス）  
4. MouthfeelCarbonation：口当たり・炭酸感（泡・ガス強度・舌触り）  
5. DrinkabilityAftertaste：飲みやすさ・後味（ゴクゴク・キレ・残留感）  
6. ABV：アルコール度数への言及（例: “0.00%”）  
7. CaloriesCarbs：カロリー・糖質関係の言及  
8. AdditivesAllergen：添加物・香料・アレルゲンの言及  
9. Ingredients：原材料・素材・産地・自然由来  
10. PackagingDesign：包装・ラベル・外装デザイン  
11. PackServingFormat：容量・パッケージ形態（缶・瓶・パック数）  
12. ShelfLifeStorage：保存性・賞味期限・開封後品質  
13. PriceValue：価格・コスパ・値ごろ感  
14. Availability：流通性・店舗・EC 販売可能性  
15. PromotionVisibility：販促・広告・POP・訴求力  
16. BrandImageStory：ブランドイメージ・理念・物語性  
17. SocialAcceptability：運転中・妊娠中などでも飲める許容感  
18. SustainabilityEthics：環境配慮・サステナビリティ・倫理性';
COMMENT ON COLUMN gold_survey_responses_final.negative_feedback_confidence IS 'negative_feedback_category に対するConfidence Score。分類信頼度（0.0～1.0）';

In [0]:
%sql
DESCRIBE hiroshi.survey_analysis_simple.gold_survey_responses_final